In [1]:
#http://wittawat.com/posts/log-sum_exp_underflow.html
#https://www.inspiredacoustics.com/en/MIDI_note_numbers_and_center_frequencies
import numpy as np

In [2]:
# Data Structures
note_names = ['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B']
midi_num = 12 # C0
midi2note = {}
note2midi = {}
for octave in range(9):
    for note in note_names:
        key = note + str(octave)
        midi2note[midi_num] = key
        note2midi[key] = midi_num
        midi_num += 1
        
interval_names = ['P1', 'm2', 'M2', 'm3', 'M3', 'P4', 'd5', 'P5', 'm6', 'M6', 'm7', 'M7', 'P8', 
                  'm9', 'M9', 'm10', 'M10']
interval2num = {}
num2interval = {}
for num, interval in enumerate(interval_names):
    interval2num[interval] = num
    num2interval[num] = interval

harmonic_table = {'P1': 0.0,
                  'm2': 0.0,
                  'M2': 0.0,
                  'm3': 0.1428,
                  'M3': 0.1428,
                  'P4': 0.0,
                  'd5': 0.0,
                  'P5': 0.1428,
                  'm6': 0.1428,
                  'M6': 0.1428,
                  'm7': 0.0,
                  'M7': 0.0,
                  'P8': 0.0004,
                  'm9': 0.0,
                  'M9': 0.0,
                  'm10': 0.1428,
                  'M10': 0.1428}

melodic_interval_names = ['m2u', 'M2u', 'm3u', 'M3u', 'P4u', 'P5u', 'm6u', 'P8u', 
                          'm2d', 'M2d', 'm3d', 'M3d', 'P4d', 'P5d', 'P8d', 'P1']
melodic_interval_inds = {n: i for i, n in enumerate(melodic_interval_names)}
melodic_interval2num = {}
melodic_num2interval = {}
for k, v in zip(melodic_interval_names, [1, 2, 3, 4, 5, 7, 8, 12, -1, -2, -3, -4, -5, -7, -12, 0]):
    melodic_interval2num[k] = v
    melodic_num2interval[v] = k
    
melodic_table = {
    'm2u': [0.0, 0.45, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0, 0.035, 0.035, 0.025, 0.025, 0.01, 0.01, 0.009, 0.001],
    'M2u': [0.45, 0.45, 0.03, 0.03, 0.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.005, 0.005, 0.004, 0.004, 0.002, 0.001],
    'm3u': [0.45, 0.45, 0.03, 0.03, 0.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.005, 0.005, 0.004, 0.004, 0.002, 0.001],
    'M3u': [0.35, 0.35, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.025, 0.025, 0.025, 0.025, 0.024, 0.001],
    'P4u': [0.065, 0.065, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.4, 0.02, 0.02, 0.01, 0.01, 0.009, 0.001],
    'P5u': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.52, 0.52, 0.01, 0.01, 0.01, 0.01, 0.009, 0.001],
    'm6u': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.51, 0.51, 0.025, 0.025, 0.01, 0.01, 0.009, 0.001],
    'P8u': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.51, 0.51, 0.025, 0.025, 0.01, 0.01, 0.009, 0.001],
    'm2d': [0.05, 0.05, 0.005, 0.005, 0.002, 0.002, 0.002, 0.002, 0.0, 0.467, 0.2, 0.2, 0.015, 0.0, 0.0, 0.001],
    'M2d': [0.05, 0.05, 0.005, 0.005, 0.002, 0.002, 0.002, 0.002, 0.367, 0.367, 0.1, 0.1, 0.015, 0.0, 0.0, 0.001],
    'm3d': [0.366, 0.366, 0.005, 0.005, 0.002, 0.002, 0.002, 0.002, 0.05, 0.05, 0.1, 0.1, 0.0, 0.0, 0.0, 0.001],
    'M3d': [0.366, 0.366, 0.005, 0.005, 0.002, 0.002, 0.002, 0.002, 0.05, 0.05, 0.1, 0.1, 0.0, 0.0, 0.0, 0.001],
    'P4d': [0.43, 0.43, 0.02, 0.02, 0.02, 0.02, 0.039, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001],
    'P5d': [0.454, 0.454, 0.03, 0.03, 0.01, 0.005, 0.005, 0.011, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001],
    'P8d': [0.454, 0.454, 0.03, 0.03, 0.01, 0.005, 0.005, 0.011, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001],
    'P1': [0.1, 0.1, 0.08, 0.08, 0.05, 0.05, 0.04, 0.03, 0.1, 0.1, 0.08, 0.08, 0.05, 0.04, 0.02, 0.0]
}
for k in melodic_interval_names: # some rows don't sum to 1
    melodic_table[k] = list(np.array(melodic_table[k])/sum(melodic_table[k]))


In [3]:
cf = ['D4', 'F4', 'G4', 'A4', 'G4', 'F4', 'E4', 'D4']
cf_pitch = [note2midi[n] for n in cf]

class Path:
    def __init__(self, pitches, score, melody):
        self.path = pitches
        self.score = score
        self.last_melody = melody
    
# first step: get candidates
# assume we are above the c.f.
start_pitch = cf_pitch[0]
paths = []
for interval, p in harmonic_table.items():
    if p > 0:
        paths.append(Path([start_pitch + interval2num[interval]], np.log(p), 'P1'))
        #print(start_pitch + interval2num[interval], np.log(p), 'P1')

# next step: for each note in melodic_interval_names find the path in that results in max score
for step in range(1, len(cf)):
    new_paths = []
    ref_pitch = cf_pitch[step]
    for interval, p in harmonic_table.items():
        if p > 0:
            cand_pitch = ref_pitch + interval2num[interval]
            #print("candidate: ", cand_pitch, interval)
            candidates = []
            scores = []
            for path in paths:
                prev_pitch = path.path[-1]
                melodic_interval = cand_pitch - prev_pitch
                if melodic_interval not in melodic_num2interval:
                    continue
                interval_name = melodic_num2interval[melodic_interval]
                index = melodic_interval_inds[interval_name]
                score = p * melodic_table[path.last_melody][index]


                new_path = path.path.copy()
                new_path.append(cand_pitch)
                new_score = path.score + np.log(score)
                candidates.append(Path(new_path, new_score, interval_name))
                scores.append(new_score)
                #print(new_path, new_score, interval_name)
            new_paths.append(candidates[np.argmax(scores)])
    paths = new_paths.copy()
for p in paths:
    print(p.path, p.score)
    print([midi2note[n] for n in p.path])


[66, 68, 70, 72, 71, 69, 67, 65] -26.977038869607618
['F#/Gb4', 'G#/Ab4', 'A#/Bb4', 'C5', 'B4', 'A4', 'G4', 'F4']
[66, 68, 70, 72, 71, 69, 68, 66] -26.67128321978837
['F#/Gb4', 'G#/Ab4', 'A#/Bb4', 'C5', 'B4', 'A4', 'G#/Ab4', 'F#/Gb4']
[66, 68, 70, 72, 74, 73, 71, 69] -26.708364894692902
['F#/Gb4', 'G#/Ab4', 'A#/Bb4', 'C5', 'D5', 'C#/Db5', 'B4', 'A4']
[66, 68, 70, 72, 74, 73, 71, 70] -26.708364894692902
['F#/Gb4', 'G#/Ab4', 'A#/Bb4', 'C5', 'D5', 'C#/Db5', 'B4', 'A#/Bb4']
[66, 68, 70, 72, 76, 74, 73, 71] -27.8306702956183
['F#/Gb4', 'G#/Ab4', 'A#/Bb4', 'C5', 'E5', 'D5', 'C#/Db5', 'B4']
[66, 68, 70, 72, 76, 74, 73, 74] -35.942712329638695
['F#/Gb4', 'G#/Ab4', 'A#/Bb4', 'C5', 'E5', 'D5', 'C#/Db5', 'D5']
[78, 80, 82, 84, 83, 81, 79, 77] -26.977038869607618
['F#/Gb5', 'G#/Ab5', 'A#/Bb5', 'C6', 'B5', 'A5', 'G5', 'F5']
[78, 80, 82, 84, 83, 81, 80, 78] -26.67128321978837
['F#/Gb5', 'G#/Ab5', 'A#/Bb5', 'C6', 'B5', 'A5', 'G#/Ab5', 'F#/Gb5']


<ipython-input-3-620ca88167ff>:41: RuntimeWarning: divide by zero encountered in log
  new_score = path.score + np.log(score)
